In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate

from langchain_community.example_selectors import NGramOverlapExampleSelector
from langchain.prompts import PromptTemplate,FewShotPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(dotenv_path=find_dotenv(filename="D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
examples=[
    {"input":"See Spot Run.","output":"Ver correr a Spot."},
    {"input": "My dog barks.", "output": "Mi perro ladra."},
    {"input": "Spot can run.", "output": "Spot puede correr."}   
]

In [7]:
examplePrompt=PromptTemplate(
    template="Input:{input}\n Output:{output}",
    input_variables=["input","output"]
)

In [9]:
exampleSelector=NGramOverlapExampleSelector(
    examples=examples,
    example_prompt=examplePrompt,
    # For negative threshold:
    # Selector sorts examples by ngram overlap score, and excludes none.
    # For threshold greater than 1.0:
    # Selector excludes all examples, and returns an empty list.
    # For threshold equal to 0.0:
    # Selector sorts examples by ngram overlap score,
    # and excludes those with no ngram overlap with input.
    threshold=-1
)
    

In [11]:
dynamicPrompt=FewShotPromptTemplate(
    example_prompt=examplePrompt,
    example_selector=exampleSelector,
    prefix="Give the Spanish translation of every input",
    suffix="English: {inputword}\nSpanish: ",
    input_variables=["inputword"]
)

In [12]:
chain=RunnablePassthrough()|dynamicPrompt|llmGemini|StrOutputParser()

In [15]:
print(chain.invoke(input={"inputword":"Spot can Run Fast"}))

Spanish: Spot puede correr rápido. 



In [16]:
exampleSelector.add_example(
    example={"input": "Spot plays fetch.", "output": "Spot juega a buscar."}
                           )

In [18]:
exampleSelector.threshold=0.03

In [20]:
print(dynamicPrompt.format(inputword="Spot can run fast."))

Give the Spanish translation of every input

Input:Spot can run.
 Output:Spot puede correr.

Input:See Spot Run.
 Output:Ver correr a Spot.

Input:Spot plays fetch.
 Output:Spot juega a buscar.

English: Spot can run fast.
Spanish: 


In [21]:
print(chain.invoke(input={"inputword":"Spot can Run Fast"}))

Spanish: Spot puede correr rápido. 

